In [1]:
import pandas as pd
import glob
import os
from datetime import datetime
import calendar

In [2]:
# Define the folder path (adjust as needed)
folder_path = './interval/'  # e.g., './data/', or 'C:/my_folder/'
# Use glob to find all .csv files
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
# Check if any files found
if not csv_files:
    print("No CSV files found.")
else:
    # Read each CSV file and store in a list
    dfs = []
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            dfs.append(df)
            print(f"Loaded: {file}")
        except Exception as e:
            print(f"Error reading {file}: {e}")
    # Concatenate all DataFrames
    df = pd.concat(dfs, ignore_index=True)
    print(f"\nTotal rows after combining: {len(df)}")
df = df.drop(columns=["ESTIMATED_ACTUAL","REVISION_DATE", "CONSUMPTION_SURPLUSGENERATION"])
df['USAGE_START_TIME'] = df['USAGE_START_TIME'].str.strip()
df['USAGE_END_TIME'] = df['USAGE_END_TIME'].str.strip()
df['hour'] = pd.to_datetime(df['USAGE_START_TIME'], format='%H:%M').dt.hour

# Convert USAGE_DATE to datetime
df['datetime'] = pd.to_datetime(df['USAGE_DATE'], format='%m/%d/%Y')
# Create 15-min interval offsets (0, 15, 30, 45 minutes)
# Assuming rows are ordered chronologically
df['datetime'] += pd.to_timedelta(df['hour'], unit='hour')
# Now you have full timestamps
# print(df[['USAGE_DATE', 'USAGE_KWH', 'datetime']])


Loaded: ./interval/IntervalData.csv
Loaded: ./interval/IntervalData (4).csv
Loaded: ./interval/IntervalData (3).csv
Loaded: ./interval/IntervalData (2).csv

Total rows after combining: 34948


In [3]:
# df

In [4]:
# Set datetime as index for resampling
df.set_index('datetime', inplace=True)
# Resample and sum kWh
hourly = df['USAGE_KWH'].resample('h').sum()
daily = df['USAGE_KWH'].resample('D').sum()
weekly = df['USAGE_KWH'].resample('W').sum()
monthly = df['USAGE_KWH'].resample('ME').sum()
# date =df['USAGE_DATE']

In [5]:
summary_df = pd.DataFrame({
    'hourly_kwh': hourly,
    'daily_kwh': daily,
    'weekly_kwh': weekly,
    'monthly_kwh': monthly
}).reset_index()
# Rename index to 'datetime'
summary_df.rename(columns={'index': 'datetime'}, inplace=True)

# Extract frequency labels from datetime
df.reset_index(inplace=True)
df['hour'] = df['datetime'].dt.hour
df['day'] = df['datetime'].dt.date
df['week'] = df['datetime'].dt.to_period('W')
df['month'] = df['datetime'].dt.to_period('M')
# Group by hour/day/week/month and aggregate
hourly_agg = df.groupby('hour')['USAGE_KWH'].sum().rename('hourly_kwh')
daily_agg = df.groupby('day')['USAGE_KWH'].sum().rename('daily_kwh')
weekly_agg = df.groupby('week')['USAGE_KWH'].sum().rename('weekly_kwh')
monthly_agg = df.groupby('month')['USAGE_KWH'].sum().rename('monthly_kwh')
# Merge back using map
df['hourly_kwh'] = df['hour'].map(hourly_agg)
df['daily_kwh'] = df['day'].map(daily_agg)
df['weekly_kwh'] = df['week'].map(weekly_agg)
df['monthly_kwh'] = df['month'].map(monthly_agg)

In [6]:
def is_month_end(date_input):
    # accepts date_input as datetime.date, datetime.datetime, or "YYYY-MM-DD" str
    if isinstance(date_input, str):
        date = datetime.strptime(date_input, "%Y-%m-%d").date()
    elif isinstance(date_input, datetime):
        date = date_input.date()
    else:
        date = date_input  # assume datetime.date
    last_day = calendar.monthrange(date.year, date.month)[1]
    return date.day == last_day

In [7]:
# summary_df = pd.DataFrame({
#     'hourly_kwh': [0.51, 0.48, 0.62],
#     'daily_kwh': [18.7, 19.2, 17.9],
#     'weekly_kwh': [125.4, 130.1, 122.8],
#     'monthly_kwh': [567.2, 571.0, 563.5]
# })
# Set datetime as index (if not already)
# summary_df.index = pd.to_datetime(datetime)
# summary_df.rename_axis('datetime', inplace=True)

                       # Reset index to make datetime a column
# summary_df = summary_df.reset_index()
# Extract date and time
summary_df['time'] = summary_df['datetime'].dt.strftime('%H:%M')
summary_df['date'] = summary_df['datetime'].dt.strftime('%Y-%m-%d')


In [8]:
# summary_df

In [9]:
# print(df.sort_values('datetime', ascending=False).head(10))

In [10]:
rythm_peak = ['18:00','19:00','20:00','21:00']


In [11]:
df = summary_df
df['hour'] = df['datetime'].dt.hour
df['day'] = df['datetime'].dt.day
df['month'] = df['datetime'].dt.month
df['year'] = df['datetime'].dt.year
df['is_month_end']= df['datetime'].dt.is_month_end

In [12]:
# df[df['is_month_end'] == True][df['monthly_kwh'] >0]

In [13]:
def process(summary_df, peaks):
    df = summary_df
    df['hour'] = df['datetime'].dt.hour
    df['day'] = df['datetime'].dt.day
    df['month'] = df['datetime'].dt.month
    df['year'] = df['datetime'].dt.year
    df['is_month_end']= df['datetime'].dt.is_month_end
    
    # Ensure date and time are properly parsed
    # df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    # Extract hour from time
    # df['hour'] = df['datetime'].dt.hour
    # Define peak hours
    
    # Create peak flag
    df['is_peak'] = df['hour'].isin(peak_hours)
    # Separate peak and non-peak usage
    df['peak_kwh'] = df['hourly_kwh'].where(df['is_peak'], 0)
    df['non_peak_kwh'] = df['hourly_kwh'].where(~df['is_peak'], 0)
    # Set datetime as index for resampling
    df.set_index('datetime', inplace=True)
    # Daily sums
    df_daily = df.resample('D').agg({
        'peak_kwh': 'sum',
        'non_peak_kwh': 'sum'
    }).rename(columns={'peak_kwh': 'daily_peak_kwh', 'non_peak_kwh': 'daily_non_peak_kwh'})
    # Weekly sums
    df_weekly = df.resample('W-MON').agg({
        'peak_kwh': 'sum',
        'non_peak_kwh': 'sum'
    }).rename(columns={'peak_kwh': 'weekly_peak_kwh', 'non_peak_kwh': 'weekly_non_peak_kwh'})
    # Monthly sums
    df_monthly = df.resample('ME').agg({
        'peak_kwh': 'sum',
        'non_peak_kwh': 'sum'
    }).rename(columns={'peak_kwh': 'monthly_peak_kwh', 'non_peak_kwh': 'monthly_non_peak_kwh'})
    # Merge back into original DataFrame using the index
    df.reset_index(inplace=True)
    df = pd.merge(df, df_daily[['daily_peak_kwh', 'daily_non_peak_kwh']], 
                  left_on='datetime', right_index=True, how='left')
    df = pd.merge(df, df_weekly[['weekly_peak_kwh', 'weekly_non_peak_kwh']], 
                  left_on='datetime', right_index=True, how='left')
    df = pd.merge(df, df_monthly[['monthly_peak_kwh', 'monthly_non_peak_kwh']], 
                  left_on='datetime', right_index=True, how='left')
    # Fill NaNs with 0 (for first few days/weeks/months with no data)
    df.fillna(0, inplace=True)
    # Optional: Drop temporary columns if needed
    # df.drop(columns=['hour', 'is_peak', 'datetime'], inplace=True)
    return df

In [14]:
peak_hours = [18, 19, 20, 21]  # Convert to integers for comparison
rythm = process(summary_df, peak_hours)
rythm_monthly = rythm[rythm['is_month_end'] == True][rythm['monthly_kwh'] >0]
rythm_monthly = rythm_monthly[["datetime","monthly_kwh", "date", "day", "monthly_peak_kwh", "monthly_non_peak_kwh"]]
rythm_monthly.to_csv('output/rythm_monthly.csv', index=False)
rythm_monthly

/var/folders/8g/jn8cz2bj1hg4ydc3f57xd6m40000gn/T/ipykernel_61544/1174549103.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rythm_monthly = rythm[rythm['is_month_end'] == True][rythm['monthly_kwh'] >0]


,datetime,monthly_kwh,date,day,monthly_peak_kwh,monthly_non_peak_kwh
720,2025-01-31,1573.465,2025-01-31,31,258.958,1314.507
1392,2025-02-28,1240.875,2025-02-28,28,210.310,1030.565
2136,2025-03-31,1281.801,2025-03-31,31,240.481,1041.320
2856,2025-04-30,1593.343,2025-04-30,30,317.652,1275.691
3600,2025-05-31,2531.349,2025-05-31,31,483.899,2047.450
4320,2025-06-30,3203.906,2025-06-30,30,579.257,2624.649
5064,2025-07-31,3525.918,2025-07-31,31,618.776,2907.142
5808,2025-08-31,3708.744,2025-08-31,31,709.796,2998.948
6528,2025-09-30,2913.173,2025-09-30,30,528.188,2384.985
7272,2025-10-31,2391.564,2025-10-31,31,389.062,2002.502


In [15]:
peak_hours = [18, 19, 20, 21]  # Convert to integers for comparison
rythm = process(summary_df, peak_hours)
rythm_daily = rythm[rythm['daily_kwh'] >0]
rythm_daily = rythm_daily[["datetime","daily_kwh", "date", "day", "daily_peak_kwh", "daily_non_peak_kwh"]]
rythm_daily.to_csv('output/rythm_daily.csv', index=False)
rythm_daily

,datetime,daily_kwh,date,day,daily_peak_kwh,daily_non_peak_kwh
0,2025-01-01,76.787,2025-01-01,1,11.985,64.802
24,2025-01-02,68.709,2025-01-02,2,10.453,58.256
48,2025-01-03,58.808,2025-01-03,3,8.559,50.249
72,2025-01-04,61.140,2025-01-04,4,6.108,55.032
96,2025-01-05,60.837,2025-01-05,5,10.215,50.622
...,...,...,...,...,...,...
8616,2025-12-26,51.099,2025-12-26,26,6.322,44.777
8640,2025-12-27,67.708,2025-12-27,27,8.736,58.972
8664,2025-12-28,55.136,2025-12-28,28,8.187,46.949
8688,2025-12-29,59.713,2025-12-29,29,18.287,41.426


In [16]:
peak_hours = [7, 8, 9, 10,11,12,13,14,15,16,17,18,19,20] 
non_peak = [21,22,23,0,1,2,3,4,5,6]
night_free_monthly = process(summary_df, peak_hours)
night_free_monthly = night_free_monthly[night_free_monthly['is_month_end'] == True][night_free_monthly['monthly_kwh'] >0]
night_free_monthly = night_free_monthly[["datetime","monthly_kwh", "date", "day", "monthly_peak_kwh", "monthly_non_peak_kwh"]]
night_free_monthly.to_csv('output/night_free_monthly.csv', index=False)
night_free_monthly

/var/folders/8g/jn8cz2bj1hg4ydc3f57xd6m40000gn/T/ipykernel_61544/1404146127.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  night_free_monthly = night_free_monthly[night_free_monthly['is_month_end'] == True][night_free_monthly['monthly_kwh'] >0]


,datetime,monthly_kwh,date,day,monthly_peak_kwh,monthly_non_peak_kwh
720,2025-01-31,1573.465,2025-01-31,31,964.192,609.273
1392,2025-02-28,1240.875,2025-02-28,28,767.130,473.745
2136,2025-03-31,1281.801,2025-03-31,31,797.502,484.299
2856,2025-04-30,1593.343,2025-04-30,30,969.254,624.089
3600,2025-05-31,2531.349,2025-05-31,31,1697.995,833.354
4320,2025-06-30,3203.906,2025-06-30,30,2115.001,1088.905
5064,2025-07-31,3525.918,2025-07-31,31,2202.926,1322.992
5808,2025-08-31,3708.744,2025-08-31,31,2343.880,1364.864
6528,2025-09-30,2913.173,2025-09-30,30,1787.525,1125.648
7272,2025-10-31,2391.564,2025-10-31,31,1506.933,884.631


In [26]:
peak_hours = [7, 8, 9, 10,11,12,13,14,15,16,17,18,19,20] 
non_peak = [21,22,23,0,1,2,3,4,5,6]
night_free_daily = process(summary_df, peak_hours)
night_free_daily = night_free_daily[night_free_daily['daily_kwh'] >0]
night_free_daily = night_free_daily[["datetime","daily_kwh", "date", "day", "daily_peak_kwh", "daily_non_peak_kwh"]]
night_free_daily.to_csv('output/night_free_daily.csv', index=False)
night_free_daily

OSError: Cannot save file into a non-existent directory: 'target'

In [18]:
# rythm

In [19]:
# rythm.drop(columns=["hourly_kwh","daily_kwh", "time", "hour", "is_peak", "peak_kwh", "non_peak_kwh", "daily_peak_kwh", "daily_non_peak_kwh", weekly_peak_kwh", "weekly_non_peak_kwh"])

In [20]:
# rythm = rythm[["datetime","monthly_kwh", "date", "day", "monthly_peak_kwh", "monthly_non_peak_kwh"]]

In [21]:
# rythm

In [22]:
# rythm.to_csv('rythm_electricity_usage_with_peaks.csv', index=False)


In [23]:
# df['hour'] = pd.to_datetime(df['USAGE_START_TIME'], format=' %H:%M').dt.hour


In [24]:
# df

In [25]:
# df_monthly